In [1]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
const PSI = PowerSimulations
using PowerSystemCaseBuilder
using Dates
using HiGHS

In [2]:
sys = build_system(PSISystems, "modified_RTS_GMLC_DA_sys")

┌ Info: Building new system modified_RTS_GMLC_DA_sys from raw data
│   sys_descriptor.raw_data = /projects/emco4286/.julia/artifacts/5098f357bad765bfefcff58f080818863ca776bd/RTS-GMLC-0.2.2
└ @ PowerSystemCaseBuilder /projects/emco4286/.julia/packages/PowerSystemCaseBuilder/dJGb8/src/build_system.jl:71
┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /projects/emco4286/.julia/packages/PowerSystems/AHyDB/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /projects/emco4286/.julia/packages/PowerSystems/AHyDB/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /projects/emco4286/.julia/packages/PowerSystems/AHyDB/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /projects/emco4286/.julia/packages/PowerSystems/AHyDB/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /projects/emco4286/.

Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,501
Type,Count
ACBus,73
Arc,109
Area,3


In [ ]:

template_uc = template_unit_commitment(network=NetworkModel(CopperPlatePowerModel, use_slacks = true))
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)
problem = DecisionModel(template_uc, sys; optimizer = solver, horizon = Hour(24))

output_dir = joinpath("/projects", "emco4286", "data", "sienna_data", "output")
build!(problem; output_dir=output_dir)
solve!(problem, export_problem_results = true)